In [14]:
from flask import Flask, request, render_template, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

app = Flask(__name__)

# Load the trained model
model = load_model('enhanced_cnn_mango_leaf_disease.h5')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# List of disease names corresponding to class indices
disease_names = [
    'Anthracnose', 
    'Bacterial Canker',
    'Cutting Weevil',
    'Die Back',
    'Gall Midge',
    'Healthy',
    'Powdery Mildew', 
    'Sooty Mold'
]


In [11]:
# Function to get treatment information based on the disease name
def get_treatment_info(disease_name):
    treatments = {
        'Anthracnose': {
            "drug": "Chlorothalonil or Copper-based fungicides",
            "procedures": "Apply fungicides at 10-15 day intervals, especially during wet seasons. Ensure thorough coverage of the leaves and fruit.",
            "duration": "Continue treatment from flowering until the fruit is harvested."
        },
        'Bacterial Canker': {
            "drug": "Copper-based sprays",
            "procedures": "Apply copper sprays during early stages of the disease. Prune infected branches and ensure good air circulation.",
            "duration": "Apply every 7-14 days depending on the severity of the infection."
        },
        'Cutting Weevil': {
            "drug": "Chlorpyrifos or Carbaryl",
            "procedures": "Apply insecticides at the base of the tree and affected areas. Monitor for weevil activity and apply as necessary.",
            "duration": "Apply during the weevil's active period, typically in the early growing season."
        },
        'Die Back': {
            "drug": "Carbendazim or Copper Oxychloride",
            "procedures": "Apply fungicides to the affected areas. Prune and destroy affected branches.",
            "duration": "Treat immediately upon detection and continue as necessary."
        },
        'Gall Midge': {
            "drug": "Imidacloprid or Abamectin",
            "procedures": "Spray insecticides during the early stages of infestation. Ensure proper coverage of the affected areas.",
            "duration": "Apply every 10-14 days during active infestation."
        },
        'Healthy': {
            "drug": "None",
            "procedures": "Maintain regular monitoring and good agricultural practices.",
            "duration": "Ongoing."
        },
        'Powdery Mildew': {
            "drug": "Sulfur-based fungicides or Potassium bicarbonate",
            "procedures": "Apply fungicides at the first sign of disease. Ensure thorough coverage of leaves.",
            "duration": "Apply every 7-10 days during the growing season."
        },
        'Sooty Mold': {
            "drug": "Horticultural oil or Neem oil",
            "procedures": "Apply oil sprays to affected areas. Ensure good coverage and repeat applications as necessary.",
            "duration": "Apply every 7-14 days until the mold is controlled."
        }
    }
    
    return treatments.get(disease_name, {
        "drug": "Unknown",
        "procedures": "No procedures available.",
        "duration": "Unknown."
    })

# Preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0, 1]
    return img_array


In [12]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file provided"})
        
        file = request.files['file']
        
        if file.filename == '':
            return jsonify({"error": "No file selected"})
        
        file_path = os.path.join('uploads', file.filename)
        file.save(file_path)

        # Preprocess the image
        preprocessed_image = preprocess_image(file_path)

        # Predict the class
        predictions = model.predict(preprocessed_image)
        predicted_class = np.argmax(predictions, axis=1)[0]
        predicted_disease = disease_names[predicted_class]

        # Get the treatment information for the predicted disease
        treatment_info = get_treatment_info(predicted_disease)

        return jsonify({
            "disease": predicted_disease,
            "drug": treatment_info['drug'],
            "procedures": treatment_info['procedures'],
            "duration": treatment_info['duration']
        })
    
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(debug=True, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1